In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
# all genre

In [4]:
html_page = requests.get('https://www.the-numbers.com/market/genres')
soup = BeautifulSoup(html_page.content,'html.parser')

In [5]:
table=soup.find('table')


In [17]:
df=pd.read_html(str(table))[0]
df

,Rank,Genre,Movies,Total Box Office,Tickets,Share
0,1,Adventure,1064,"$63,772,981,564",9132914210,27.33%
1,2,Action,1027,"$47,442,579,508",6731964309,20.33%
2,3,Drama,5303,"$37,244,278,625",5707951137,15.96%
3,4,Comedy,2322,"$33,968,678,469",5233596160,14.56%
4,5,Thriller/Suspense,1088,"$19,522,115,970",2848524119,8.37%
5,6,Horror,602,"$11,745,747,045",1697739525,5.03%
6,7,Romantic Comedy,594,"$9,918,998,969",1599171071,4.25%
7,8,Musical,183,"$4,105,500,177",542118782,1.76%
8,9,Documentary,2319,"$2,237,329,848",336008169,0.96%
9,10,Black Comedy,187,"$1,662,498,025",231336380,0.71%


In [7]:
list(df.itertuples(index=False,name=None))

[(1, 'Adventure', 1064, '$63,772,981,564', 9132914210, '27.33%'),
 (2, 'Action', 1027, '$47,442,579,508', 6731964309, '20.33%'),
 (3, 'Drama', 5303, '$37,244,278,625', 5707951137, '15.96%'),
 (4, 'Comedy', 2322, '$33,968,678,469', 5233596160, '14.56%'),
 (5, 'Thriller/Suspense', 1088, '$19,522,115,970', 2848524119, '8.37%'),
 (6, 'Horror', 602, '$11,745,747,045', 1697739525, '5.03%'),
 (7, 'Romantic Comedy', 594, '$9,918,998,969', 1599171071, '4.25%'),
 (8, 'Musical', 183, '$4,105,500,177', 542118782, '1.76%'),
 (9, 'Documentary', 2319, '$2,237,329,848', 336008169, '0.96%'),
 (10, 'Black Comedy', 187, '$1,662,498,025', 231336380, '0.71%'),
 (11, 'Western', 81, '$1,084,785,441', 149931395, '0.46%'),
 (12, 'Concert/Performance', 74, '$395,155,838', 53881827, '0.17%'),
 (13, 'Multiple Genres', 42, '$33,688,583', 4071085, '0.01%'),
 (14, 'Reality', 4, '$621,696', 73810, '0.00%')]

In [8]:
# individual genre

In [9]:
all_df = []

for genre in df['Genre']:
    genre_url='https://www.the-numbers.com/market/genre/'+genre.replace('/','-or-').replace(' ','-')
    res=requests.get(genre_url)
    soup=BeautifulSoup(res.text,'html.parser')
    tables=soup.find_all('table')
    df2=pd.read_html(str(tables[1]))[0]
    df2['Genre']=genre
    all_df.append(df2)

In [18]:
all_df[1]

,Year,Movies inRelease,MarketShare,Gross,Tickets Sold,Inflation-AdjustedGross,Top-Grossing Movie,Gross that Year,Genre
0,1995,29,19.75%,"$1,049,544,278",241274531,"$2,198,010,977",Batman Forever,"$184,031,112",Action
1,1996,36,21.19%,"$1,222,675,745",276623456,"$2,520,039,684",Twister,"$241,688,385",Action
2,1997,36,22.91%,"$1,456,612,580",317344773,"$2,891,010,882",The Lost World: Jurassic Park,"$229,086,679",Action
3,1998,30,14.37%,"$973,161,753",207497162,"$1,890,299,146",Rush Hour,"$137,567,454",Action
4,1999,32,12.42%,"$911,259,780",179381832,"$1,634,168,490",The Matrix,"$171,383,253",Action
5,2000,31,16.82%,"$1,266,568,598",234984879,"$2,140,712,248",Mission: Impossible 2,"$215,409,889",Action
6,2001,32,16.70%,"$1,395,248,989",246510408,"$2,245,709,817",Rush Hour 2,"$226,164,286",Action
7,2002,32,15.22%,"$1,394,290,142",239981075,"$2,186,227,593",Men in Black 2,"$190,418,803",Action
8,2003,36,22.95%,"$2,109,404,900",349818376,"$3,186,845,405",The Matrix Reloaded,"$281,553,689",Action
9,2004,28,11.51%,"$1,068,864,943",172119948,"$1,568,012,726","I, Robot","$144,801,023",Action


In [12]:
# Most Profitable Movies, Based on Absolute Profit on Worldwide Gross

In [13]:
html_page = requests.get('https://www.the-numbers.com/movie/budgets')
soup = BeautifulSoup(html_page.content,'html.parser')

In [32]:
tables=soup.find_all('table')

In [36]:
df=pd.read_html(str(tables[2]))[0]

In [38]:
df.dropna()

,Unnamed: 0,Release Date,Movie,Approx. Income,Approx. Expense,Profit
0,1.0,"Dec 18, 2009",Avatar,"$1,796,427,464","$328,262,000","$1,468,165,464"
2,2.0,"Apr 26, 2019",Avengers: Endgame,"$1,479,816,843","$475,560,000","$1,004,256,843"
4,3.0,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,"$1,224,827,707","$381,704,000","$843,123,707"
6,4.0,"Nov 22, 2013",Frozen,"$1,042,311,056","$245,904,000","$796,407,056"
8,5.0,"Nov 18, 2005",Harry Potter and the Goblet of Fire,"$997,569,652","$208,064,000","$789,505,652"
10,6.0,"Apr 27, 2018",Avengers: Infinity War,"$1,125,651,663","$368,812,000","$756,839,663"
12,7.0,"May 4, 2012",The Avengers,"$998,554,456","$300,290,000","$698,264,456"
14,8.0,"Jun 12, 2015",Jurassic World,"$962,588,221","$287,756,000","$674,832,221"
16,9.0,"Jul 15, 2011",Harry Potter and the Deathly Hallows: Part II,"$833,252,878","$177,306,000","$655,946,878"
18,10.0,"Jul 3, 2013",Despicable Me 2,"$735,831,589","$143,056,000","$592,775,589"


In [ ]:
# Biggest Money Losers, Based on Absolute Loss on Worldwide Earnings

In [39]:
html_page = requests.get('https://www.the-numbers.com/movie/budgets')
soup = BeautifulSoup(html_page.content,'html.parser')

In [ ]:
tables=soup.find_all('table')

In [42]:
df=pd.read_html(str(tables[4]))[0]

In [44]:
df.dropna()

,Unnamed: 0,Release Date,Movie,Approx. Income,Approx. Expense,Profit
0,1.0,"Mar 11, 2011",Mars Needs Moms,"$26,754,640","$170,166,000","-$143,411,360"
2,2.0,"Mar 9, 2012",John Carter,"$180,857,835","$295,824,000","-$114,966,165"
4,3.0,"May 12, 2017",King Arthur: Legend of the Sword,"$84,936,081","$199,580,000","-$114,643,919"
6,4.0,"Sep 30, 2016",Deepwater Horizon,"$78,201,830","$189,348,000","-$111,146,170"
8,5.0,"Jan 13, 2017",Monster Trucks,"$39,953,396","$149,450,000","-$109,496,604"
10,6.0,"Mar 1, 2013",Jack the Giant Slayer,"$123,521,590","$228,504,000","-$104,982,410"
12,7.0,"Dec 17, 2010",How Do You Know?,"$35,567,429","$140,454,000","-$104,886,571"
14,8.0,"Dec 25, 2013",47 Ronin,"$93,230,564","$189,988,000","-$96,757,436"
16,9.0,"Feb 6, 2015",Jupiter Ascending,"$110,744,962","$206,582,000","-$95,837,038"
18,10.0,"Apr 21, 2017",The Promise,"$6,231,500","$99,886,000","-$93,654,500"


In [40]:
# import pandas as pd
# from bs4 import BeautifulSoup
# soup = BeautifulSoup('html_page', 'lxml') # Parse the HTML as a string
# table = soup.find_all('table')[0] # Grab the first table
# new_table = pd.DataFrame(columns=range(0,2), index = [0]) # I know the size 
# row_marker = 0
# for row in table.find_all('tr'):
#     column_marker = 0
#     columns = row.find_all('td')
#     for column in columns:
#         new_table.iat[row_marker,column_marker] = column.get_text()
#         column_marker += 1
    
# new_table

In [41]:
# adventure_movies = soup.find('div',{'id':'page_filling_chart'})
# tables=adventure_movies.find_all('table')

In [14]:
# data=[]

# rows=tables[0].findAll('tr')
# for tr in rows:
#     cols=tr.findAll('td')
#     text_tuple = tuple([td.text.replace('\n','').replace('\xa0','').replace(' ','') for td in cols])
#     data.append(text_tuple)



In [17]:
# import pandas as pd
# a=pd.DataFrame(data)

In [19]:
# a.head()